In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
groq_api_key =  os.getenv("GROQ_API_KEY")
groq_api_key

In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Anand and I'm a prompt Engineer")])

In [ ]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, my name is Anand and I'm a prompt Engineer"),
        AIMessage(content="Hi Anand, it's nice to meet you! \n\nThat's a fascinating field! I'm always interested in learning more about how people interact with and guide large language models like me.\n\nWhat kind of projects are you working on as a prompt engineer?  Do you have a favorite type of prompt to create?\n"),
        HumanMessage(content="Hey, what's my name and what do I do ?")
    ]
)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
config={"configurable": {"session_id": "chat1"}}

In [ ]:
with_message_history=RunnableWithMessageHistory(model, get_session_history=get_session_history)

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Anand and I'm a prompt Engineer")],
    config=config
)

In [ ]:
response.content

In [ ]:
with_message_history.invoke(
    [HumanMessage(content="What's my name ?")],
    config=config
)

In [ ]:
config1={"configurable": {"session_id": "chat2"}}

response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)

In [ ]:
response.content

# Prompt template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best for you ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model


In [ ]:
chain.invoke({"messages":[HumanMessage(content="Hi, My name is Anand.")]})

In [ ]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "chat3"}}

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Anand")],
    config=config
)

In [ ]:
response

# Add more complexity

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your knowledge in {language}"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [ ]:
response = chain.invoke({"messages":[HumanMessage(content="Hi My name is Anand")], "language": "Hindi"})
print(response.content)

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [ ]:
config={"configurable": {"session_id": "chat4"}}
response=with_message_history.invoke(
    { "messages": [HumanMessage(content="Hi, I'm Anand")], "language": "Hindi"},
        config=config
)

response.content

# Managing Conversation History

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [ ]:
messages = [
    SystemMessage(content="you're a good Assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response=chain.invoke(
    {
        "messages":messages + [HumanMessage(content="what ice cream do I like")],
        "language": "English"
    }
)

response.content

In [ ]:
response=chain.invoke(
    {
        "messages":messages + [HumanMessage(content="what math problem did I ask for")],
        "language": "English"
    }
)

response.content

# Let's Warp this in the message history

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config5={"configurable": {"session_id": "chat5"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name")],
        "language": "English",
    },
    config=config5
)

response.content

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did I ask for")],
        "language": "English",
    },
    config=config5
)

response.content